
# Projet 2

In [1]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [2]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [41]:
#Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.

geo['CP Ville'] = geo['Code Postal']+' '+geo['Commune']

geo['CP Ville']

Code INSEE
01001    01400 L'ABERGEMENT-CLEMENCIAT
01002      01640 L'ABERGEMENT-DE-VAREY
01004          01500 AMBERIEU-EN-BUGEY
01005        01330 AMBERIEUX-EN-DOMBES
01006                    01300 AMBLEON
01007                   01500 AMBRONAY
01008                   01500 AMBUTRIX
01009           01300 ANDERT-ET-CONDON
                     ...              
97610                    97600 KOUNGOU
97611                  97600 MAMOUDZOU
97612                  97630 MTSAMBORO
97613              97650 M'TSANGAMOUJI
97614                   97670 OUANGANI
97615                   97610 PAMANDZI
97616                       97640 SADA
97617                   97680 TSINGONI
Name: CP Ville, Length: 36742, dtype: object

In [37]:
# fonction recherche de ville
def ville(lat, long):
    geo['Distance'] =np.sqrt((geo['Latitude']-lat)**2+(geo['Longitude']-long)**2)
    
    city = geo['Commune'].loc[geo['Distance'].idxmin()]
    return city

#ville(46.153426,4.926114) 



"L'ABERGEMENT-CLEMENCIAT"

In [38]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + mn / 60 + sec / 3600

In [39]:
# on applique la fonction à une coordonnée tirée au hasard
# init du random
np.random.seed(0)

a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'BOSJEAN'

In [75]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [76]:
# Donner le décompte des valeurs accédées avec la clé "type".
import json

def get_json_field(json_value, field): 
    return json.loads(json_value)[field]

# geo['geo_shape_type'] = [json.loads(geo_shape)['type']  for geo_shape in geo["geo_shape"].values] trop lourd 

geo['geo_shape_type'] = geo['geo_shape'].apply(get_json_field,field='type')


geo['geo_shape_type'].value_counts()

Polygon         36670
MultiPolygon       72
Name: geo_shape_type, dtype: int64

In [79]:
#Donner le décompte des longueurs des listes accédées avec la clé "coordinates".

geo['geo_shape_coordinates_length'] = geo['geo_shape'].apply(get_json_field,field='coordinates').apply(len)


geo['geo_shape_coordinates_length'].value_counts()

1    36660
2       80
3        1
4        1
Name: geo_shape_coordinates_length, dtype: int64

In [82]:
#Quelle commune est la plus complexe géométriquement ?
Commune = geo['Commune'].loc[geo['geo_shape_coordinates_length'].idxmax()]

Commune

'HYERES'

In [83]:
#Quelle commune est la seconde la plus complexe géométriquement ?
geo['Commune'].loc[geo['geo_shape_coordinates_length']==3]

Code INSEE
27701    VAL-DE-REUIL
Name: Commune, dtype: object

In [89]:
#Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
cities = geo['Commune'].loc[(geo['geo_shape_type']=='Polygon') & (geo['geo_shape_coordinates_length']==2)]
cities

Code INSEE
02232               COYOLLES
05052               EYGLIERS
09042    LA BASTIDE-DE-SEROU
22209              PLOUBALAY
2A272                SARTENE
2B049              CALENZANA
48080               LANGOGNE
67486              SUNDHOUSE
68078              EGUISHEIM
71028            BEAUVERNOIS
Name: Commune, dtype: object

In [104]:
#Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.
from shapely.geometry import Polygon
for city in cities: 
    coordinates=geo['geo_shape'].apply(get_json_field,field='coordinates').loc[(geo['Commune']==city)].values[0]
    
    print("{} --Le premier polygone contient le second(enclave): {}".format(Polygon(coordinates[0]).contains(Polygon(coordinates[1]))))
    


Le premier polygone contient le second(enclave): True
[[3.037897560658703, 49.16921270775551], [3.022651486402025, 49.170816855142384], [3.016004215046372, 49.17997922335811], [2.991714837684887, 49.19196418704024], [2.98137479741421, 49.1844740569323], [2.971813659157835, 49.187528930723204], [2.969496165186601, 49.20994111330158], [2.970866610714616, 49.21304305603823], [2.95873860742351, 49.22660554534713], [2.964349025205721, 49.231911431076426], [3.001619992136874, 49.23565651032947], [3.013095594217706, 49.229218996200984], [3.008290195644244, 49.22338489957146], [3.017032136166848, 49.21975199592734], [3.026996333441546, 49.22011876947888], [3.039328911223897, 49.23046290137869], [3.033310889856941, 49.243338284278906], [3.040777749481918, 49.239928826200895], [3.04971736146969, 49.24144476338487], [3.055118307185106, 49.24608118447443], [3.055004734193956, 49.24258418221722], [3.064571289474025, 49.23964873656758], [3.072714287073868, 49.23075304901955], [3.052139968025862, 49.

Le premier polygone contient le second(enclave): True
[[3.850531831651621, 44.68745054452225], [3.841797319218708, 44.68992121193551], [3.835414561952019, 44.702079092055754], [3.809299991076912, 44.69887880511517], [3.81011269784627, 44.70886658913203], [3.796992491197599, 44.71762305033686], [3.7903208160594, 44.73710022848897], [3.791996159461473, 44.74664093608505], [3.805820420015999, 44.74771171118465], [3.812621618189193, 44.752361872171655], [3.825219048975053, 44.748919460501746], [3.829914990945916, 44.75463764760284], [3.839418691548341, 44.747120131946], [3.862527328127106, 44.743872437148596], [3.875635603121645, 44.73947491433879], [3.877138133617485, 44.736185735657685], [3.863182979295896, 44.71908066005584], [3.863127621411198, 44.71217543665072], [3.86992027570744, 44.70725141446061], [3.869647744005664, 44.69696264049236], [3.854383929195778, 44.69262482228565], [3.850531831651621, 44.68745054452225]]
Le premier polygone contient le second(enclave): True
[[7.59493925